In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# Configuration

### Inputs

In [2]:
process_out_dir = '01_process/out/'

train_data_fpath = process_out_dir + 'train_data.npz'
valid_data_fpath = process_out_dir + 'valid_data.npz'
# not doing any test set stuff until the very, very end

In [3]:
extended_dir = '/caldera/projects/usgs/water/iidd/datasci/lake-temp/lake_ice_prediction/'

process_out_dir = extended_dir + process_out_dir

train_data_fpath = extended_dir + train_data_fpath
valid_data_fpath = extended_dir + valid_data_fpath

### Values

In [4]:
epochs = 10000

# plotting parameters
loss_curve_zoom_ymax = 0.15
loss_curve_zoom_ymin = 0.055

### Outputs

In [5]:
train_out_dir = '02_train/out/'

data_scalars_fpath =  train_out_dir + 'limitted_nn_lstm_min_max_scalars.pt'
model_weights_fpath = train_out_dir + 'limitted_nn_lstm_weights.pth'
train_predictions_fpath = train_out_dir + 'limitted_nn_lstm_train_preds.npy'
valid_predictions_fpath = train_out_dir + 'limitted_nn_lstm_valid_preds.npy'

In [6]:
data_scalars_fpath = extended_dir + data_scalars_fpath
model_weights_fpath = extended_dir + model_weights_fpath
train_predictions_fpath = extended_dir + train_predictions_fpath
valid_predictions_fpath = extended_dir + valid_predictions_fpath
#loss_lists_fpath = extended_dir + loss_lists_fpath

# Import

In [7]:
train_data = np.load(train_data_fpath, allow_pickle = True)
valid_data = np.load(valid_data_fpath, allow_pickle = True)

In [8]:
train_data.files

['x', 'y', 'dates', 'DOW', 'features']

In [9]:
train_x = train_data['x']
train_y = train_data['y']
train_dates = train_data['dates']
train_DOW = train_data['DOW']
train_variables = train_data['features']

In [10]:
valid_x = valid_data['x']
valid_y = valid_data['y']
valid_dates = valid_data['dates']
valid_DOW = valid_data['DOW']
valid_variables = valid_data['features']

### Quick view of all the target sequences

# Prepare data for `torch`

In [11]:
train_y = torch.from_numpy(train_y).float().unsqueeze(2) # adding a feature dimension to Ys
train_x = torch.from_numpy(train_x).float()

valid_y = torch.from_numpy(valid_y).float().unsqueeze(2)
valid_x = torch.from_numpy(valid_x).float()

# min-max scale the data

In [12]:
min_max_scalars = torch.zeros(train_x.shape[2], 2)

for i in range(train_x.shape[2]):
    min_max_scalars[i, 0] = train_x[:, :, i].min()
    min_max_scalars[i, 1] = train_x[:, :, i].max()

In [13]:
for i in range(train_x.shape[2]):
    # scale train set with train min/max
    train_x[:, :, i] = ((train_x[:, :, i] - min_max_scalars[i, 0]) /
                        (min_max_scalars[i, 1] - min_max_scalars[i, 0]))
    # scale valid set with train min/max
    valid_x[:, :, i] = ((valid_x[:, :, i] - min_max_scalars[i, 0]) /
                        (min_max_scalars[i, 1] - min_max_scalars[i, 0]))

# Define a simple model

In [14]:
# recycled model code
class LSTMDA(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout = 0):
        super().__init__()
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, dropout = dropout,
                            batch_first = True) # PROJECT LEVEL DESIGN
        
        self.dense = nn.Linear(hidden_dim, 1)
        self.dense_activation = nn.Sigmoid()
        
    def forward(self, x):
        """Assumes x is of shape (batch, sequence, feature)"""
       
        lstm_out, (h, c) = self.lstm(x)
        out = self.dense_activation(self.dense(lstm_out))
        
        return out

In [15]:
# initialize the model with a seed
torch.manual_seed(0)

# very small model
# maps 13 variables to hidden dim of 1 via LSTM layer
# transforms that LSTM out with a dense layer (scale and bias)
# then sigmoid activation for probability
model = LSTMDA(11, 1).cuda()

# Training

### Train loop

In [16]:
%%time

loss_fn = torch.nn.BCELoss()
loss_ls = []
valid_loss_ls = []

optimizer = torch.optim.Adam(model.parameters())
for i in range(epochs):
    train_y_hat = model(train_x.cuda())
    loss = loss_fn(train_y_hat, train_y.cuda())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_ls.append(loss.item())
    
    if i % int(epochs / 10) == 0:
        print(i, loss.item())
        
    with torch.no_grad():
        valid_y_hat = model(valid_x.cuda())
        valid_loss = loss_fn(valid_y_hat, valid_y.cuda())
        valid_loss_ls.append(valid_loss.item())
        
print(epochs, loss.item())

0 0.6655966639518738
1000 0.24757108092308044
2000 0.15258921682834625
3000 0.10675746202468872
4000 0.08650056272745132
5000 0.07551487535238266
6000 0.06892869621515274
7000 0.064667709171772
8000 0.06175287067890167
9000 0.06013138219714165
10000 0.05909085273742676
CPU times: user 6min 5s, sys: 23.3 s, total: 6min 29s
Wall time: 6min 44s


# Save predictions for evaluation

To save on file size, I'm not going to rebundle the other objects, they can be combined later with a simple concatenate

In [18]:
train_y_hat = model(train_x.cuda())
valid_y_hat = model(valid_x.cuda())

In [19]:
train_y_hat = train_y_hat.detach().cpu().numpy()
valid_y_hat = valid_y_hat.detach().cpu().numpy()

In [20]:
np.save(train_predictions_fpath, train_y_hat)
np.save(valid_predictions_fpath, valid_y_hat)

# Save model weights and min-max scalars

In [21]:
torch.save(min_max_scalars, data_scalars_fpath)
torch.save(model.state_dict(), model_weights_fpath)

In [24]:
for i in np.arange(0, 10000, 1000):
    print(int(i), valid_loss_ls[int(i)])
valid_loss_ls[-1]

0 0.6610182523727417
1000 0.24859009683132172
2000 0.15261214971542358
3000 0.10815349221229553
4000 0.08829427510499954
5000 0.07753383368253708
6000 0.0711306557059288
7000 0.06706433743238449
8000 0.06418035924434662
9000 0.06259708851575851


0.06161261349916458